# Guided Project: Predicting House Sale Prices
## Compiled By: Dana Hagist

In this course, we started by building intuition for model based learning, explored how the linear regression model worked, understood how the two different approaches to model fitting worked, and some techniques for cleaning, transforming, and selecting features. In this guided project, you can practice what you learned in this course by exploring ways to improve the models we built.

You'll work with housing data for the city of Ames, Iowa, United States from 2006 to 2010. You can read more about why the data was collected here (http://ww2.amstat.org/publications/jse/v19n3/decock.pdf). You can also read about the different columns in the data here (https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt).

Let's start by setting up a pipeline of functions that will let us quickly iterate on different models.

You can find the solutions notebook for this guided project here (https://github.com/dataquestio/solutions/blob/master/Mission240Solutions.ipynb).

Task(s):

- Import pandas, matplotlib, and numpy into the environment. Import the classes you need from scikit-learn as well.
- Read AmesHousing.tsv into a pandas data frame.
- For the following functions, we recommend creating them in the first few cells in the notebook. This way, you can add cells to the end of the notebook to do experiments and update the functions in these cells.
    - Create a function named transform_features() that, for now, just returns the train data frame.
    - Create a function named select_features() that, for now, just returns the Gr Liv Area and SalePrice columns from the train data frame.
    - Create a function named train_and_test() that, for now:
        - Selects the first 1460 rows from from data and assign to train.
        - Selects the remaining rows from data and assign to test.
        - Trains a model using all numerical columns except the SalePrice column (the target column) from the data frame returned from select_features()
        - Tests the model on the test set and returns the RMSE value.


In [110]:
# Solution:
import pandas as pd
pd.options.display.max_columns = 999
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

In [111]:
# Reading in dataframe
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")

In [112]:
def transform_features(df):
    return df

def select_features(df):
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df):
    train = df[0:1460]
    test = df[1460:]
    
    # Using pd.dataframe.select_dtypes to only includes 
    # columns with certain datatypes.
    train_numeric = train.select_dtypes(include = ['float','integer'])
    test_numeric = test.select_dtypes(include = ['float', 'integer'])
    
    # Using pd.drop to drop a value
    features = train_numeric.columns.drop('SalePrice')
    
    # Using target column for solution extensibility
    target = 'SalePrice'
    
    # Creating Linear Regression instance:
    lr = LinearRegression()
    lr.fit(train[features], train[target])
    
    # Predicting on test set
    predictions = lr.predict(test[features])
    
    # Calculating errors (MSE and RMSE)
    mse = mean_squared_error(test[target], predictions)
    rmse = np.sqrt(mse)
    #rmse = mse**(1/2) Alternative square-root calculation

    return rmse

# Using above functions to process dataframe and return RMSE
transform_df = transform_features(df)
features_df = select_features(transform_df)
rmse = train_and_test(features_df)

# Printing rmse from above
print(rmse)

57088.25161263909


## Feature Engineering
Let's now start removing features with many missing values, diving deeper into potential categorical features, and transforming text and numerical columns. Update transform_features() so that any column from the data frame with more than 25% (or another cutoff value) missing values is dropped. You also need to remove any columns that leak information about the sale (e.g. like the year the sale happened). In general, the goal of this function is to:

- remove features that we don't want to use in the model, just based on the number of missing values or data leakage
- transform features into the proper format (numerical to categorical, scaling numerical, filling in missing values, etc)
- create new features by combining other features

Next, you need to get more familiar with the remaining columns by reading the data documentation (https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt) for each column, determining what transformations are necessary (if any), and more. As we mentioned earlier, succeeding in predictive modeling (and competitions like Kaggle) is highly dependent on the quality of features the model has. Libraries like scikit-learn has made it quick and easy to simply try and tweak many different models, but cleaning, selecting, and transforming features is still more of an art that requires a bit of human ingenuity.

Task(s):
- As we mentioned earlier, we recommend adding some cells to explore and experiment with different features (before rewriting these functions).
- The transform_features() function shouldn't modify the train data frame and instead return a new one entirely. This way, we can keep using train in the experimentation cells.
- Which columns contain less than 5% missing values?
    - For numerical columns that meet this criteria, let's fill in the missing values using the most popular value for that column.
- What new features can we create, that better capture the information in some of the features?
    - An example of this would be the years_until_remod feature we created in the last mission.
- Which columns need to be dropped for other reasons?
    - Which columns aren't useful for machine learning?
    - Which columns leak data about the final sale?

In [116]:
# Solution(s):

# Find number of missing values, sorted by most missing
missing_count = df.isnull().sum().sort_values(ascending = False)
missing_count.head()

Pool QC         2917
Misc Feature    2824
Alley           2732
Fence           2358
Fireplace Qu    1422
dtype: int64

In [114]:
# Create variable for columns with more than 5% missing
high_missing = missing_count[(missing_count/len(df) > .05)]
print(high_missing.sort_values(ascending=False))

# Dropping these columns from the dataframe
# In this case index is returning the label, which we are dropping
# axis = 1 refers to the column axis in the dataframe
df = df.drop(high_missing.index, axis = 1)

Pool QC          2917
Misc Feature     2824
Alley            2732
Fence            2358
Fireplace Qu     1422
Lot Frontage      490
Garage Finish     159
Garage Cond       159
Garage Yr Blt     159
Garage Qual       159
Garage Type       157
dtype: int64


In [74]:
print(df.head())

   Order        PID  MS SubClass MS Zoning  Lot Area Street Lot Shape  \
0      1  526301100           20        RL     31770   Pave       IR1   
1      2  526350040           20        RH     11622   Pave       Reg   
2      3  526351010           20        RL     14267   Pave       IR1   
3      4  526353030           20        RL     11160   Pave       Reg   
4      5  527105010           60        RL     13830   Pave       IR1   

  Land Contour Utilities Lot Config Land Slope Neighborhood Condition 1  \
0          Lvl    AllPub     Corner        Gtl        NAmes        Norm   
1          Lvl    AllPub     Inside        Gtl        NAmes       Feedr   
2          Lvl    AllPub     Corner        Gtl        NAmes        Norm   
3          Lvl    AllPub     Corner        Gtl        NAmes        Norm   
4          Lvl    AllPub     Inside        Gtl      Gilbert        Norm   

  Condition 2 Bldg Type House Style  Overall Qual  Overall Cond  Year Built  \
0        Norm      1Fam      1S

In [75]:
# Dropping text columns with more than 1 missing value

# Finding text columns with missing values
text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)

# Filter Series to columns containing *any* missing values
drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]

df = df.drop(drop_missing_cols_2.index, axis=1)

In [80]:
# Filling in missing numeric values with most popular for that column

# Finding missing numeric values
numeric_missing = df.select_dtypes(include=['integer', 'float']).isnull().sum().sort_values(ascending = False)

# Finding columns between 1 missing value and 5% missing values
fixable_numeric_cols = numeric_missing[(numeric_missing < len(df)/20) & (numeric_missing > 0)].sort_values(ascending = False)
fixable_numeric_cols

Mas Vnr Area      23
Bsmt Full Bath     2
Bsmt Half Bath     2
Garage Area        1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
BsmtFin SF 1       1
Garage Cars        1
dtype: int64

In [81]:
# Compute the most common value for each column in `fixable_numeric_missing_cols`.
replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
replacement_values_dict

{'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Bsmt Unf SF': 0.0,
 'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Garage Area': 0.0,
 'Garage Cars': 2.0,
 'Mas Vnr Area': 0.0,
 'Total Bsmt SF': 0.0}

In [83]:
# Fill values in fixable numeric missing columns from above
df = df.fillna(replacement_values_dict)

In [98]:
# Check again for missing values
df.isnull().sum().value_counts()

0    64
dtype: int64

In [104]:
# Creating feature for age of home at sale
home_age = df['Yr Sold'] - df['Year Built']

# Checking for unexpected (negative values)
home_age[home_age < 0]

2180   -1
dtype: int64

In [106]:
# Creating features for age of remodel at sale
remod_age = df['Yr Sold'] - df['Year Remod/Add']

# Checking for negative values
remod_age[remod_age < 0]

1702   -1
2180   -2
2181   -1
dtype: int64

In [108]:
# Adding the above new features to the dataframe

# Adding home age feature
df['home_age'] = years_sold

# Adding remodel age feature
df['remod_age'] = remod_age

# Drop rows with negative values for both of these new features
df = df.drop([1702, 2180, 2181], axis=0)

# Drop original year columns
df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)

In [109]:
# Drop columns that aren't useful for ML
df = df.drop(["PID", "Order"], axis=1)

# Drop columns that leak info about the final sale
df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

In [115]:
# Updating the transform_features function
# See above cells for commentary on each of these updates

def transform_features(df):
    missing_count = df.isnull().sum().sort_values(ascending = False)
    high_missing = missing_count[(missing_count/len(df) > .05)]
    df = df.drop(high_missing.index, axis = 1)
    
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols_2.index, axis=1)
    
    numeric_missing = df.select_dtypes(include=['integer', 'float']).isnull().sum().sort_values(ascending = False)
    fixable_numeric_cols = numeric_missing[(numeric_missing < len(df)/20) & (numeric_missing > 0)].sort_values(ascending = False)
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    home_age = df['Yr Sold'] - df['Year Built']
    
    remod_age = df['Yr Sold'] - df['Year Remod/Add']
    
    df['home_age'] = years_sold


    df['remod_age'] = remod_age

    df = df.drop([1702, 2180, 2181], axis=0)

    df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)
 
    df = df.drop(["PID", "Order"], axis=1)

    df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
        
    return df

def select_features(df):
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df):
    train = df[0:1460]
    test = df[1460:]
    
    # Using pd.dataframe.select_dtypes to only includes 
    # columns with certain datatypes.
    train_numeric = train.select_dtypes(include = ['float','integer'])
    test_numeric = test.select_dtypes(include = ['float', 'integer'])
    
    # Using pd.drop to drop a value
    features = train_numeric.columns.drop('SalePrice')
    
    # Using target column for solution extensibility
    target = 'SalePrice'
    
    # Creating Linear Regression instance:
    lr = LinearRegression()
    lr.fit(train[features], train[target])
    
    # Predicting on test set
    predictions = lr.predict(test[features])
    
    # Calculating errors (MSE and RMSE)
    mse = mean_squared_error(test[target], predictions)
    rmse = np.sqrt(mse)
    #rmse = mse**(1/2) Alternative square-root calculation

    return rmse

# Reading in fresh dataframe
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")

# Using above functions to process dataframe and return RMSE
transform_df = transform_features(df)
features_df = select_features(transform_df)
rmse = train_and_test(features_df)

# Printing rmse from above
print(rmse)

55275.36731241307


Results: Can see based on the above that we were able to reduce a good amount of our RMSE by doing some of this data cleaning and feature engineering, reinforcing how important these steps are.  Same model, better results.

## Numeric Feature Selection
Now that we have cleaned and transformed a lot of the features in the data set, it's time to move on to feature selection for numerical features.

Task(s):
- Generate a correlation heatmap matrix of the numerical features in the training data set.
    - Which features correlate strongly with our target column, SalePrice?
    - Calculate the correlation coefficients for the columns that seem to correlate well with SalePrice. Because we have a pipeline in place, it's easy to try different features and see which features result in a better cross validation score.
- Which columns in the data frame should be converted to the categorical data type? All of the columns marked as nominal from the documentation (https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt) are candidates for being converted to categorical. Here are some other things you should think about:
    - If a categorical column has hundreds of unique values (or categories), should you keep it? When you dummy code this column, hundreds of columns will need to be added back to the data frame.
    - Which categorical columns have a few unique values but more than 95% of the values in the column belong to a specific category? This would be similar to a low variance numerical feature (no variability in the data for the model to capture)
- Which columns are currently numerical but need to be encoded as categorical instead (because the numbers don't have any semantic meaning)?
- What are some ways we can explore which categorical columns "correlate" well with SalePrice?
    - Read this post for some potential strategies (http://machinelearningmastery.com/feature-selection-machine-learning-python/).
- Update the logic for the select_features() function. This function should take in the new, modified train and test data frames that were returned from transform_features().

In [120]:
# Solution(s):

# Creating dataframe of numeric columns
numerical_df = transform_df.select_dtypes(include = ['integer', 'float'])
numerical_df.head()

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,home_age,remod_age
0,20,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,0,0,1656,1.0,0.0,1,0,3,1,7,2,2.0,528.0,210,62,0,0,0,0,0,215000,50,50
1,20,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,0,0,896,0.0,0.0,1,0,2,1,5,0,1.0,730.0,140,0,0,0,120,0,0,105000,49,49
2,20,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,0,0,1329,0.0,0.0,1,1,3,1,6,0,1.0,312.0,393,36,0,0,0,0,12500,172000,52,52
3,20,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,0,0,2110,1.0,0.0,2,1,3,1,8,2,2.0,522.0,0,0,0,0,0,0,0,244000,42,42
4,60,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,701,0,1629,0.0,0.0,2,1,3,1,6,1,2.0,482.0,212,34,0,0,0,0,0,189900,13,12


In [122]:
# Looking at correlation between feature columns and sale price
abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values(ascending = False)
# Viewing these coefficients in descending order
abs_corr_coeffs

SalePrice          1.000000
Overall Qual       0.801206
Gr Liv Area        0.717596
Garage Cars        0.648361
Total Bsmt SF      0.644012
Garage Area        0.641425
1st Flr SF         0.635185
home_age           0.558979
Full Bath          0.546118
remod_age          0.534985
Mas Vnr Area       0.506983
TotRms AbvGrd      0.498574
Fireplaces         0.474831
BsmtFin SF 1       0.439284
Wood Deck SF       0.328183
Open Porch SF      0.316262
Half Bath          0.284871
Bsmt Full Bath     0.276258
2nd Flr SF         0.269601
Lot Area           0.267520
Bsmt Unf SF        0.182751
Bedroom AbvGr      0.143916
Enclosed Porch     0.128685
Kitchen AbvGr      0.119760
Screen Porch       0.112280
Overall Cond       0.101540
MS SubClass        0.085128
Pool Area          0.068438
Low Qual Fin SF    0.037629
Bsmt Half Bath     0.035875
3Ssn Porch         0.032268
Misc Val           0.019273
BsmtFin SF 2       0.006127
Name: SalePrice, dtype: float64

In [124]:
# Keeping columns with a correlation coefficient of larger than 0.4 
# This is worth revisiting to try other thresholds
abs_corr_coeffs[abs_corr_coeffs > 0.4]

SalePrice        1.000000
Overall Qual     0.801206
Gr Liv Area      0.717596
Garage Cars      0.648361
Total Bsmt SF    0.644012
Garage Area      0.641425
1st Flr SF       0.635185
home_age         0.558979
Full Bath        0.546118
remod_age        0.534985
Mas Vnr Area     0.506983
TotRms AbvGrd    0.498574
Fireplaces       0.474831
BsmtFin SF 1     0.439284
Name: SalePrice, dtype: float64

In [129]:
# Dropping columns that don't meet the above correlation threshold
transform_df = transform_df.drop(abs_corr_coeffs[abs_corr_coeffs < 0.4].index, axis = 1)

In [131]:
# Viewing updated dataframe
transform_df.head()

,MS Zoning,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,BsmtFin SF 1,Total Bsmt SF,Heating,Heating QC,Central Air,1st Flr SF,Gr Liv Area,Full Bath,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Garage Cars,Garage Area,Paved Drive,SalePrice,home_age,remod_age
0,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,Hip,CompShg,BrkFace,Plywood,112.0,TA,TA,CBlock,639.0,1080.0,GasA,Fa,Y,1656,1656,1,TA,7,Typ,2,2.0,528.0,P,215000,50,50
1,RH,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,Gable,CompShg,VinylSd,VinylSd,0.0,TA,TA,CBlock,468.0,882.0,GasA,TA,Y,896,896,1,TA,5,Typ,0,1.0,730.0,Y,105000,49,49
2,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,Hip,CompShg,Wd Sdng,Wd Sdng,108.0,TA,TA,CBlock,923.0,1329.0,GasA,TA,Y,1329,1329,1,Gd,6,Typ,0,1.0,312.0,Y,172000,52,52
3,RL,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,Hip,CompShg,BrkFace,BrkFace,0.0,Gd,TA,CBlock,1065.0,2110.0,GasA,Ex,Y,2110,2110,2,Ex,8,Typ,2,2.0,522.0,Y,244000,42,42
4,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,Gable,CompShg,VinylSd,VinylSd,0.0,TA,TA,PConc,791.0,928.0,GasA,Gd,Y,928,1629,2,TA,6,Typ,1,2.0,482.0,Y,189900,13,12


In [133]:
# Creating list of column names from documentation that are meant to be categorical
nominal_features = ["Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
"Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
"Misc Feature", "Sale Type", "Sale Condition"]

In [141]:
# Creating list of categorical columns
transform_cat_cols = []
for col in nominal_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)

# Calculating unique values in each cateorical column
uniqueness_counts = transform_df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values(ascending = False)


# Aribtrary cutoff of 10 unique values (worth experimenting)
drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index

# Dropping the above non-unique columns
transform_df = transform_df.drop(drop_nonuniq_cols, axis=1)
print(transform_df.head())

  MS Zoning Street Lot Shape Land Contour Utilities Lot Config Land Slope  \
0        RL   Pave       IR1          Lvl    AllPub     Corner        Gtl   
1        RH   Pave       Reg          Lvl    AllPub     Inside        Gtl   
2        RL   Pave       IR1          Lvl    AllPub     Corner        Gtl   
3        RL   Pave       Reg          Lvl    AllPub     Corner        Gtl   
4        RL   Pave       IR1          Lvl    AllPub     Inside        Gtl   

  Neighborhood Condition 1 Condition 2 Bldg Type House Style  Overall Qual  \
0        NAmes        Norm        Norm      1Fam      1Story             6   
1        NAmes       Feedr        Norm      1Fam      1Story             5   
2        NAmes        Norm        Norm      1Fam      1Story             6   
3        NAmes        Norm        Norm      1Fam      1Story             7   
4      Gilbert        Norm        Norm      1Fam      2Story             5   

  Roof Style Roof Matl  Mas Vnr Area Exter Qual Exter Cond Foundatio

In [142]:
# Convert reamaining text columns to categorical
text_cols = transform_df.select_dtypes(include=['object'])
for col in text_cols:
    transform_df[col] = transform_df[col].astype('category')
    
# Creating dummy columns and add back to the dataframe
transform_df = pd.concat([
    transform_df, 
    pd.get_dummies(transform_df.select_dtypes(include=['category']))
], axis=1)

## Training and Testing with Cross-Validation
Now for the final part of the pipeline, training and testing. When iterating on different features, using simple validation is a good idea. Let's add a parameter named k that controls the type of cross validation that occurs.

Task(s):
- The optional k parameter should accept integer values, with a default value of 0.
- When k equals 0, perform holdout validation (what we already implemented):
    - Select the first 1460 rows and assign to train.
    - Select the remaining rows and assign to test.
    - Train on train and test on test.
    - Compute the RMSE and return.
- When k equals 1, perform simple cross validation:
    - Shuffle the ordering of the rows in the data frame.
    - Select the first 1460 rows and assign to fold_one.
    - Select the remaining rows and assign to fold_two.
    - Train on fold_one and test on fold_two.
    - Train on fold_two and test on fold_one.
    - Compute the average RMSE and return.
- When k is greater than 0, implement k-fold cross validation using k folds:
    - Perform k-fold cross validation using k folds.
    - Calculate the average RMSE value and return this value.

In [159]:
# Updating the transform_features function
# See above cells for commentary on each of these updates

def transform_features(df):
    missing_count = df.isnull().sum().sort_values(ascending = False)
    high_missing = missing_count[(missing_count/len(df) > .05)]
    df = df.drop(high_missing.index, axis = 1)
    
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(drop_missing_cols_2.index, axis=1)
    
    numeric_missing = df.select_dtypes(include=['integer', 'float']).isnull().sum().sort_values(ascending = False)
    fixable_numeric_cols = numeric_missing[(numeric_missing < len(df)/20) & (numeric_missing > 0)].sort_values(ascending = False)
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    home_age = df['Yr Sold'] - df['Year Built']
    
    remod_age = df['Yr Sold'] - df['Year Remod/Add']
    
    df['home_age'] = years_sold

    df['remod_age'] = remod_age

    df = df.drop([1702, 2180, 2181], axis=0)

    df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)
 
    df = df.drop(["PID", "Order"], axis=1)

    df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
        
    return df

def select_features(df, coeff_threshold=0.4, uniq_threshold=10):
    numerical_df = df.select_dtypes(include=['int', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1)
    
    return df

def train_and_test(df, k=0):
    # Creating numeric df in order to select features below
    numeric_df = df.select_dtypes(include=['integer', 'float'])

    # Using pd.drop to drop a value
    features = numeric_df.columns.drop('SalePrice')

    # Creating Linear Regression instance:
    lr = LinearRegression()
    
    # Using target column for solution extensibility
    target = 'SalePrice'
    
    if k == 0:
        train = df[:1460]
        test = df[1460:]

        # Fitting regression model
        lr.fit(train[features], train[target])
        
        # Predicting on test set
        predictions = lr.predict(test[features])
        
        # Calculating errors (MSE and RMSE)
        mse = mean_squared_error(test[target], predictions)
        rmse = np.sqrt(mse)

        return rmse
  

    if k == 1:
        # Randomize *all* rows (frac=1) from `df` and return
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train[target])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test[target], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test[target])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train[target], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train[target])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test[target], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse

# Reading in fresh dataframe
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")

# Using above functions to process dataframe and return RMSE
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=5)

# Printing rmse from above
print(rmse)

# Iterating over k-values to analyze different results
for i in range(1,6):
    rmse = train_and_test(filtered_df, k=i)
    print(rmse)

[39257.063453268354, 27069.666018864187, 28078.88717674864, 24577.235382925166, 25891.240229645864]
28974.818452290445
33367.287183402834
27155.138484824703
30261.21283411377
[25952.461007174745, 32231.193365393887]
29091.827186284318
[27314.1371726412, 25988.69951793576, 34942.88043456576]
29415.23904171424
[28755.662198738926, 36781.88981875915, 26493.499835594, 23702.03918185537]
28933.272758736864
[25899.84621126231, 37749.37187111981, 28757.06909508466, 25826.085518368043, 26662.839866119084]
28979.04251239078


Appears based on the above that a k-value around 4 is likely a suitable value for this model, although more iterations would have to be run to have any certainty around that.

## Possible Extensions to the Project

- Continue iteration on feature engineering:
    - Reserach some other approaches to feature engineering online around housing data.
    - Visit the Kaggle kernels page for this dataset to see approaches others took.
- Improve your feature selection:
- Research ways of doing feature selection better with categorical columns (something we didn't cover in this particular course).